In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
#Checking if gpu is being used
#tf.debugging.set_log_device_placement(True)

# Create some tensors
#a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
#b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
#c = tf.matmul(a, b)

#print(c)

In [2]:
# hyperparameters
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

# Set random seed
tf.random.set_seed(1337)

In [3]:
#Loading data
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
text[:200]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you'

In [5]:
chars = sorted(list(set(text)))

In [7]:
len(chars)

65

In [11]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

In [ ]:
# Create character mappings

In [12]:
def encode(s): 
    return [stoi[c] for c in s]

In [13]:
def decode(l): 
    return ''.join([itos[i] for i in l])

In [14]:
encode('this is a cow')

[58, 46, 47, 57, 1, 47, 57, 1, 39, 1, 41, 53, 61]

In [15]:
decode([58, 46, 47, 57, 1, 47, 57, 1, 39, 1, 41, 53, 61])

'this is a cow'

In [19]:
# Train and test splits
data = tf.constant(encode(text), dtype=tf.int64)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [29]:
ix = tf.random.uniform(shape=(batch_size,), maxval=len(train_data) - block_size, dtype=tf.int32)
ix

<tf.Tensor: shape=(64,), dtype=int32, numpy=
array([ 651260,  913423,  514658,  110057,  955054,   94418,  456445,
         17137,  957196,  733081,  659498,  677301,  377637,   81229,
        294407,  830827,  904722,  949357,  145512,  111565,  197655,
        532504,  259175,  825917,    8824,  371660,   95880,  143439,
        400292,  496633,  510513,  598725,   88834,  506673,  561963,
         71153, 1002923,  625874,  661900,  908728,  356407,  323578,
         12237,  812283,  555632,  370357,  445768,  866733,  849474,
        539309,  699931,  906447, 1001751,  287124,  199115,  283600,
        174024,  869222,  735086,  948993,  941412,  439930,  162198,
        834124], dtype=int32)>

In [20]:
# Data loading into batches
def get_batch(split):
    data_split = train_data if split == 'train' else val_data
    ix = tf.random.uniform(shape=(batch_size,), maxval=len(data_split) - block_size, dtype=tf.int32)
    x = tf.stack([data_split[i:i+block_size] for i in ix])
    y = tf.stack([data_split[i+1:i+block_size+1] for i in ix])
    return x, y

In [ ]:
sentence

In [23]:
x, y = get_batch('train')

In [25]:
x[0], y[0]

(<tf.Tensor: shape=(256,), dtype=int64, numpy=
 array([53, 59, 56,  1, 43, 57, 58, 39, 58, 43,  1, 56, 43, 55, 59, 47, 56,
        43, 57,  1, 39, 52, 42,  1, 51, 47, 52, 43,  1, 41, 39, 52,  1, 63,
        47, 43, 50, 42,  8,  0,  0, 35, 13, 30, 35, 21, 15, 23, 10,  0, 20,
        43, 52, 56, 63,  1, 52, 53, 61,  1, 50, 47, 60, 43, 57,  1, 47, 52,
         1, 31, 41, 53, 58, 50, 39, 52, 42,  1, 39, 58,  1, 46, 47, 57,  1,
        43, 39, 57, 43,  6,  0, 35, 46, 43, 56, 43,  1, 46, 39, 60, 47, 52,
        45,  1, 52, 53, 58, 46, 47, 52, 45,  6,  1, 52, 53, 58, 46, 47, 52,
        45,  1, 41, 39, 52,  1, 46, 43,  1, 50, 53, 57, 43,  8,  0, 13, 52,
        42,  1, 39, 57,  1, 44, 53, 56,  1, 63, 53, 59,  1, 63, 53, 59, 56,
        57, 43, 50, 44,  6,  1, 53, 59, 56,  1, 55, 59, 53, 52, 42, 39, 51,
         1, 55, 59, 43, 43, 52,  6,  0, 37, 53, 59,  1, 46, 39, 60, 43,  1,
        39,  1, 44, 39, 58, 46, 43, 56,  1, 39, 40, 50, 43,  1, 58, 53,  1,
        51, 39, 47, 52, 58, 39, 47, 52,  

In [26]:
decode([53, 59, 56,  1, 43, 57, 58, 39, 58, 43,  1, 56, 43, 55, 59, 47, 56,
        43, 57,  1, 39, 52, 42,  1, 51, 47, 52, 43,  1, 41, 39, 52,  1, 63,
        47, 43, 50, 42,  8,  0,  0, 35, 13, 30, 35, 21, 15, 23, 10,  0, 20,
        43, 52, 56, 63,  1, 52, 53, 61,  1, 50, 47, 60, 43, 57,  1, 47, 52,
         1, 31, 41, 53, 58, 50, 39, 52, 42,  1, 39, 58,  1, 46, 47, 57,  1,
        43, 39, 57, 43,  6,  0, 35, 46, 43, 56, 43,  1, 46, 39, 60, 47, 52,
        45,  1, 52, 53, 58, 46, 47, 52, 45,  6,  1, 52, 53, 58, 46, 47, 52,
        45,  1, 41, 39, 52,  1, 46, 43,  1, 50, 53, 57, 43,  8,  0, 13, 52,
        42,  1, 39, 57,  1, 44, 53, 56,  1, 63, 53, 59,  1, 63, 53, 59, 56,
        57, 43, 50, 44,  6,  1, 53, 59, 56,  1, 55, 59, 53, 52, 42, 39, 51,
         1, 55, 59, 43, 43, 52,  6,  0, 37, 53, 59,  1, 46, 39, 60, 43,  1,
        39,  1, 44, 39, 58, 46, 43, 56,  1, 39, 40, 50, 43,  1, 58, 53,  1,
        51, 39, 47, 52, 58, 39, 47, 52,  1, 63, 53, 59, 11,  0, 13, 52, 42,
         1, 40, 43, 58, 58, 43, 56,  1,  5, 58, 61, 43, 56, 43,  1, 63, 53,
        59,  1, 58, 56, 53, 59, 40, 50, 43, 42,  1, 46, 47, 51,  1, 58, 46,
        39])

"our estate requires and mine can yield.\n\nWARWICK:\nHenry now lives in Scotland at his ease,\nWhere having nothing, nothing can he lose.\nAnd as for you yourself, our quondam queen,\nYou have a father able to maintain you;\nAnd better 'twere you troubled him tha"

In [27]:
decode([59, 56,  1, 43, 57, 58, 39, 58, 43,  1, 56, 43, 55, 59, 47, 56, 43,
        57,  1, 39, 52, 42,  1, 51, 47, 52, 43,  1, 41, 39, 52,  1, 63, 47,
        43, 50, 42,  8,  0,  0, 35, 13, 30, 35, 21, 15, 23, 10,  0, 20, 43,
        52, 56, 63,  1, 52, 53, 61,  1, 50, 47, 60, 43, 57,  1, 47, 52,  1,
        31, 41, 53, 58, 50, 39, 52, 42,  1, 39, 58,  1, 46, 47, 57,  1, 43,
        39, 57, 43,  6,  0, 35, 46, 43, 56, 43,  1, 46, 39, 60, 47, 52, 45,
         1, 52, 53, 58, 46, 47, 52, 45,  6,  1, 52, 53, 58, 46, 47, 52, 45,
         1, 41, 39, 52,  1, 46, 43,  1, 50, 53, 57, 43,  8,  0, 13, 52, 42,
         1, 39, 57,  1, 44, 53, 56,  1, 63, 53, 59,  1, 63, 53, 59, 56, 57,
        43, 50, 44,  6,  1, 53, 59, 56,  1, 55, 59, 53, 52, 42, 39, 51,  1,
        55, 59, 43, 43, 52,  6,  0, 37, 53, 59,  1, 46, 39, 60, 43,  1, 39,
         1, 44, 39, 58, 46, 43, 56,  1, 39, 40, 50, 43,  1, 58, 53,  1, 51,
        39, 47, 52, 58, 39, 47, 52,  1, 63, 53, 59, 11,  0, 13, 52, 42,  1,
        40, 43, 58, 58, 43, 56,  1,  5, 58, 61, 43, 56, 43,  1, 63, 53, 59,
         1, 58, 56, 53, 59, 40, 50, 43, 42,  1, 46, 47, 51,  1, 58, 46, 39,
        52])

"ur estate requires and mine can yield.\n\nWARWICK:\nHenry now lives in Scotland at his ease,\nWhere having nothing, nothing can he lose.\nAnd as for you yourself, our quondam queen,\nYou have a father able to maintain you;\nAnd better 'twere you troubled him than"

In [6]:
# Calculating loss of the model
def estimate_loss(model):
    out = {}
    model.trainable = False
    for split in ['train', 'val']:
        losses = tf.TensorArray(tf.float32, size=eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses = losses.write(k, loss)
        out[split] = losses.stack().numpy().mean()
    model.trainable = True
    return out

In [7]:
class Head(tf.keras.layers.Layer):
    """ one head of self-attention """

    def __init__(self, head_size):
        super(Head, self).__init__()
        self.key = tf.keras.layers.Dense(head_size, use_bias=False)
        self.query = tf.keras.layers.Dense(head_size, use_bias=False)
        self.value = tf.keras.layers.Dense(head_size, use_bias=False)

        tril = tf.linalg.band_part(tf.ones((block_size, block_size)), -1, 0)
        self.tril = tf.constant(tril)

        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x)   # (B, T, hs)
        q = self.query(x) # (B, T, hs)

        # compute attention scores ("affinities")
        wei = tf.matmul(q, tf.transpose(k, perm=[0, 2, 1])) * tf.math.rsqrt(tf.cast(k.shape[-1], tf.float32))  # (B, T, T)
        wei = tf.where(self.tril[:T, :T] == 0, float('-inf'), wei)  # (B, T, T)
        wei = tf.nn.softmax(wei, axis=-1)  # (B, T, T)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        v = self.value(x)  # (B, T, hs)
        out = tf.matmul(wei, v)  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [8]:
class MultiHeadAttention(tf.keras.layers.Layer):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super(MultiHeadAttention, self).__init__()
        self.heads = [Head(head_size) for _ in range(num_heads)]
        self.proj = tf.keras.layers.Dense(n_embd)
        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x):
        out = tf.concat([h(x) for h in self.heads], axis=-1)
        out = self.dropout(self.proj(out))
        return out

In [9]:
class FeedForward(tf.keras.layers.Layer):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super(FeedForward, self).__init__()
        self.net = tf.keras.Sequential([
            tf.keras.layers.Dense(4 * n_embd),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dense(n_embd),
            tf.keras.layers.Dropout(dropout),
        ])

    def call(self, x):
        return self.net(x)

In [10]:
class Block(tf.keras.layers.Layer):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super(Block, self).__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.ln2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [11]:
class GPTLanguageModel(tf.keras.Model):

    def __init__(self):
        super(GPTLanguageModel, self).__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size, n_embd)
        self.position_embedding_table = tf.keras.layers.Embedding(block_size, n_embd)
        self.blocks = tf.keras.Sequential([Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.lm_head = tf.keras.layers.Dense(vocab_size, kernel_initializer='normal', bias_initializer='zeros')

    def call(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(tf.range(T, dtype=tf.float32))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = tf.reshape(logits, (B * T, C))
            targets = tf.reshape(targets, (B * T,))
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(targets, logits)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = tf.nn.softmax(logits, axis=-1)  # (B, C)
            # sample from the distribution
            idx_next = tf.random.categorical(tf.math.log(probs), num_samples=1, dtype=tf.int64)  # (B, 1)
            # append sampled index to the running sequence
            idx = tf.concat([idx, idx_next], axis=1)  # (B, T+1)
        return idx


In [ ]:
# Initializing model with pre-trained weights. Use this if you don't want to re-train the model.
#model = GPTLanguageModel()
#dummy_input = tf.constant([[0]], dtype=tf.int32)  # Example input, adjust shape as needed
#model(dummy_input)
#model.load_weights('gpt_model_weights.h5')

In [12]:
#Training the model. GPU is recommended for training.

model = GPTLanguageModel()
optimizer = tf.keras.optimizers.Adam(learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    with tf.GradientTape() as tape:
        logits, loss = model(xb, yb)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))


step 0: train loss 4.5158, val loss 4.5177
step 500: train loss 1.9006, val loss 2.0083
step 1000: train loss 1.4417, val loss 1.6584
step 1500: train loss 1.2854, val loss 1.5992
step 2000: train loss 1.1676, val loss 1.5936
step 2500: train loss 1.0419, val loss 1.6674
step 3000: train loss 0.9076, val loss 1.8094
step 3500: train loss 0.7525, val loss 2.0218
step 4000: train loss 0.6012, val loss 2.3162
step 4500: train loss 0.4598, val loss 2.6565
step 4999: train loss 0.3497, val loss 2.9876


In [13]:
# generate from the model
context = tf.zeros((1, 1), dtype=tf.int64)
generated_sequence = model.generate(context, max_new_tokens=500).numpy()
print(decode(generated_sequence[0]))


Now keeps.
Can I know should thee were trans--I protest,
To betwixt the Samart's the mutine.

CAMILLO:
Ha, madam!
Sir, you!
You pitiff now, but you are worth aboards,
Betwixt the right of your ox adversaries,
Or let our suddenly in all severaltius free
Than Bolingbroke to England. Mercutio,
Ever justice with his praisence, he was proud;
When she departed by his fortune like a greer,
And in the gentle king fair hateful man.
Farewell; so old Cominius, away; I rather,
To you are therefore be behold


In [14]:
model.save_weights('gpt_model_weights.h5')